In [1]:
import pandas as pd
import numpy as np


In [2]:
original_data = pd.read_csv('Transformed_Minedex_data.csv')
original_data.head()

,SiteCode,StartDate,EndDate,Tonnage (Mt),Grade,GradeUnit,CommodityName,ContainedMetal,ContainedMetalUnit,CutOffGrade,TargetCommodityGroup,quantity(tonne),ppm
0,S0000001,13/07/2005,14/09/2006,0.695,1.30,gram/tonne,Gold,903.50,kilogram,NaN,GOLD,0.90350,1.30
1,S0000002,12/03/2019,NaN,0.528,2.09,gram/tonne,Gold,1103.52,kilogram,0.3,GOLD,1.10352,2.09
2,S0000005,30/09/2006,07/03/2016,0.011,7.33,gram/tonne,Gold,80.63,kilogram,NaN,GOLD,0.08063,7.33
3,S0000005,30/09/2006,07/03/2016,0.018,4.94,gram/tonne,Gold,88.92,kilogram,NaN,GOLD,0.08892,4.94
4,S0000006,08/10/1993,30/12/2014,0.057,2.36,gram/tonne,Gold,134.52,kilogram,NaN,GOLD,0.13452,2.36


In [3]:
original_data.GradeUnit.unique()

array(['gram/tonne', 'percent', 'kilogram/tonne', 'carat/tonne'],
      dtype=object)

In [4]:
original_data.columns

Index(['SiteCode', 'StartDate', 'EndDate', 'Tonnage (Mt)', 'Grade',
       'GradeUnit', 'CommodityName', 'ContainedMetal', 'ContainedMetalUnit',
       'CutOffGrade', 'TargetCommodityGroup', 'quantity(tonne)', 'ppm'],
      dtype='object')

In [5]:
df=original_data[original_data['CommodityName'] == 'Nickel']

In [6]:
# Subset data with all commodities
df2 = original_data[original_data['SiteCode'].isin(list(df.SiteCode.unique()))]

In [7]:
commoditites_combination=df2.groupby('SiteCode')['CommodityName'].unique()

In [8]:
a = dict(commoditites_combination)

In [9]:
d=[]
for i in a:
    if list(a[i]) == ['Cobalt', 'Nickel'] or list(a[i]) == ['Cobalt', 'Nickel']:
        d.append(i)

In [10]:
# Vast of majority sites just get Nickel quantity

In [11]:
## Build pipeline just base on Nickel feature

In [12]:
# Subset data with Nickel and Cobalt
dataset = df2[df2['SiteCode'].isin(d)]
            

In [13]:
def filtered_data(df):

    site_minerals=[]
    for i in range(len(df.SiteCode.unique())):
        a=df[df['SiteCode'] == df.SiteCode.unique()[i]]
        g = a.groupby('CommodityName')['ppm'].sum()
        g=dict(g)
        site_minerals.append(g)
        
    row=[]
    for site in site_minerals:
        dic={}
        for i in df.CommodityName.unique():
            if i in site.keys():
                dic[i]=site[i]
            else:
                dic[i]=0
        row.append(dic)

    new_data= pd.DataFrame(row)
    new_data.insert(0, "SiteCode",  list(df.SiteCode.unique()), True)
    
    return new_data

In [14]:
# Create table of commodites for each site code (the quantities are summed up)
test_data=filtered_data(dataset)

In [15]:
# Filter dataset by minerals appear in WAMEX data
test_data.head()

,SiteCode,Cobalt,Nickel
0,S0002179,2000.0,22000.0
1,S0002181,2200.0,33000.0
2,S0002182,2400.0,21000.0
3,S0002327,400.0,7500.0
4,S0002811,700.0,12000.0


In [16]:
test_data=test_data.loc[:, ['SiteCode','Nickel','Cobalt']] 

In [17]:
Sites_data = pd.read_csv('Sites.csv')

In [18]:
Sites_data=Sites_data.loc[:,['Latitude','Longitude','SiteCode']]

In [19]:
# Add Latitude, Longitude to dataframe
test_data=pd.merge(test_data,Sites_data,on='SiteCode',how='left')

In [20]:
test_data.head()

,SiteCode,Nickel,Cobalt,Latitude,Longitude
0,S0002179,22000.0,2000.0,-30.655910,121.826202
1,S0002181,33000.0,2200.0,-30.736222,121.802770
2,S0002182,21000.0,2400.0,-30.772570,121.793602
3,S0002327,7500.0,400.0,-30.534769,122.084999
4,S0002327,7500.0,400.0,-30.534769,122.084999


In [21]:
test_data.describe()

,Nickel,Cobalt,Latitude,Longitude
count,50.000000,50.000000,50.000000,50.000000
mean,15618.400000,1156.200000,-29.715940,121.647854
std,10924.824261,873.477427,1.129027,0.641590
min,4500.000000,260.000000,-33.589151,118.674386
25%,9650.000000,600.000000,-30.444073,121.725648
50%,11100.000000,855.000000,-29.248060,121.829151
75%,19550.000000,1500.000000,-28.834288,121.915774
max,56000.000000,4400.000000,-28.062266,122.356697


## Train random forrest model on WAMEX data for unique commotities



In [22]:
train_data = pd.read_csv('Merged_Ni_lessthan15km.csv',index_col=0)
train_data.head()

,Unnamed: 0_x,Ni,Cu,Co,Pd,Au,Pt,Rh,Fe,S,Ag,lat,long,Unnamed: 0_y,minedex_coords,minedex_site,wamex_lat,wamex_lon,distance
0,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46661,"(-17.34893, 128.022095)",S0001442,-17.234642,128.032146,12.693593881734987 km
1,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46662,"(-17.31078, 128.023499)",S0002890,-17.234642,128.032146,8.476376684524638 km
2,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46663,"(-17.3526, 128.027695)",S0019720,-17.234642,128.032146,13.063277176872425 km
3,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46664,"(-17.3526, 128.027695)",S0019821,-17.234642,128.032146,13.063277176872425 km
4,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46665,"(-17.338411, 128.017007)",S0231765,-17.234642,128.032146,11.59661465949521 km


In [23]:
# Using Eli's code for precessing WAMEX dataset
df_unique = train_data.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'lat', 'long','minedex_coords', 'minedex_site','distance'])
df_unique = df_unique.drop_duplicates()

### Transform data using log transform

In [24]:
train_data = df_unique.fillna(0)

In [25]:
#Seperate dataset (exclude long and lat)
train_data=train_data.iloc[:,:-2]

In [26]:
train_data=train_data.replace(0,1)


In [27]:
train_data=train_data.iloc[:,[0,2]]

In [28]:
train_data = train_data.apply(np.log, axis=1)


In [29]:
train_data['wamex_lat'] = df_unique['wamex_lat']
train_data['wamex_lon'] = df_unique['wamex_lon']

In [30]:
train_data.head()

,Ni,Co,wamex_lat,wamex_lon
0,4.025352,2.197225,-17.234642,128.032146
5,3.433987,1.791759,-17.234642,128.032146
10,4.499810,2.708050,-17.234642,128.032146
15,3.610918,2.397895,-17.216592,128.047184
18,4.060443,2.639057,-17.216028,128.048165


In [31]:
y = train_data['Co']
X = train_data.drop(columns=['Co'])

In [32]:
X.describe()

,Ni,wamex_lat,wamex_lon
count,23233.000000,23233.000000,23233.000000
mean,4.846256,-27.805376,122.064656
std,2.051320,4.027483,3.081927
min,-7.600902,-33.697944,116.266019
25%,3.465736,-30.972928,120.517366
50%,4.605170,-28.094057,121.117641
75%,6.413459,-26.123169,122.515106
max,13.304685,-17.187316,128.993500


### Train the regression model

In [33]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [34]:
#Import Random Forest Regressor module
from sklearn.ensemble import RandomForestRegressor
# Create a based model
rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [35]:
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_test)

from sklearn.metrics import mean_squared_error

print(f"Train MSE: {mean_squared_error(train_preds, y_train)}")
print(f"Test MSE: {mean_squared_error(test_preds, y_test)}")

Train MSE: 0.08323919302531851
Test MSE: 0.5773157130891136


### Apply trained model for the Minedex data

In [36]:
original_test_data=test_data
data_pred_Co=test_data.loc[:, ['Nickel','Latitude','Longitude']] 

In [37]:
#Seperate dataset (exclude long and lat)
data_pred_Co=data_pred_Co.iloc[:,:-2]

In [38]:
data_pred_Co=data_pred_Co.replace(0,1)

In [39]:
data_pred_Co = data_pred_Co.apply(np.log, axis=1)

In [40]:
data_pred_Co['Latitude'] = original_test_data['Latitude']
data_pred_Co['Longitude'] = original_test_data['Longitude']

In [41]:
data_pred_Co.head()

,Nickel,Latitude,Longitude
0,9.998798,-30.655910,121.826202
1,10.404263,-30.736222,121.802770
2,9.952278,-30.772570,121.793602
3,8.922658,-30.534769,122.084999
4,8.922658,-30.534769,122.084999


In [42]:
test_preds = np.exp(rf.predict(data_pred_Co))

In [43]:
result=original_test_data.loc[:,['SiteCode','Cobalt']]

In [44]:
result['Pred_Cobalt'] = test_preds


In [45]:
result

,SiteCode,Cobalt,Pred_Cobalt
0,S0002179,2000.0,335.945260
1,S0002181,2200.0,723.601065
2,S0002182,2400.0,319.037878
3,S0002327,400.0,179.775792
4,S0002327,400.0,179.775792
5,S0002811,700.0,535.672977
6,S0005015,2400.0,421.611642
7,S0005039,570.0,603.926617
8,S0005040,560.0,573.322654
9,S0005041,900.0,650.494628


In [46]:
original_data[original_data['SiteCode'].isin(['S0002179'])]

,SiteCode,StartDate,EndDate,Tonnage (Mt),Grade,GradeUnit,CommodityName,ContainedMetal,ContainedMetalUnit,CutOffGrade,TargetCommodityGroup,quantity(tonne),ppm
7418,S0002179,30/06/1999,29/03/2005,2.4,0.11,percent,Cobalt,2640.0,tonnes,NaN,NICKEL,2640.0,1100.0
7419,S0002179,30/06/1999,29/03/2005,2.4,1.20,percent,Nickel,28.8,kilotonne,NaN,NICKEL,28800.0,12000.0
7420,S0002179,30/06/1999,29/03/2005,0.2,0.09,percent,Cobalt,180.0,tonnes,NaN,NICKEL,180.0,900.0
7421,S0002179,30/06/1999,29/03/2005,0.2,1.00,percent,Nickel,2.0,kilotonne,NaN,NICKEL,2000.0,10000.0


In [47]:
original_data[original_data['SiteCode'].isin(['S0005316'])]

,SiteCode,StartDate,EndDate,Tonnage (Mt),Grade,GradeUnit,CommodityName,ContainedMetal,ContainedMetalUnit,CutOffGrade,TargetCommodityGroup,quantity(tonne),ppm
8907,S0005316,30/06/2008,29/06/2013,5.86,0.065,percent,Cobalt,3809.0000,tonnes,NaN,NICKEL,3809.0,650.0
8908,S0005316,30/06/2008,29/06/2013,5.86,1.029,percent,Nickel,60.2994,kilotonne,0.8,NICKEL,60299.4,10290.0
8909,S0005316,30/06/2008,29/06/2013,9.29,0.061,percent,Cobalt,5666.9000,tonnes,NaN,NICKEL,5666.9,610.0
8910,S0005316,30/06/2008,29/06/2013,9.29,0.989,percent,Nickel,91.8781,kilotonne,0.8,NICKEL,91878.1,9890.0
8911,S0005316,30/06/2008,30/06/2013,12.97,0.061,percent,Cobalt,7911.7000,tonnes,NaN,NICKEL,7911.7,610.0
8912,S0005316,30/06/2008,30/06/2013,12.97,0.954,percent,Nickel,123.7338,kilotonne,0.8,NICKEL,123733.8,9540.0


In [48]:
# Summed up all values affects MSE result commodites ,.
# Notice that there are pairs between Cobalt and Nickel,they measured by same method, same time etc ( therefore same tonnage)
# Subset test data with minimum ppm values

In [49]:
list_of_sitecode = dataset.SiteCode.unique()


In [50]:
new_df = pd.DataFrame()
for i in dataset.SiteCode.unique():
    site=dataset[dataset['SiteCode'].str.contains(i)]
    minValue = site['Tonnage (Mt)'].min()
    final=site[site['Tonnage (Mt)']==minValue]
    new_df = new_df.append(final)
    
    

In [51]:
new_df.head()

,SiteCode,StartDate,EndDate,Tonnage (Mt),Grade,GradeUnit,CommodityName,ContainedMetal,ContainedMetalUnit,CutOffGrade,TargetCommodityGroup,quantity(tonne),ppm
7420,S0002179,30/06/1999,29/03/2005,0.2,0.09,percent,Cobalt,180.0,tonnes,NaN,NICKEL,180.0,900.0
7421,S0002179,30/06/1999,29/03/2005,0.2,1.00,percent,Nickel,2.0,kilotonne,NaN,NICKEL,2000.0,10000.0
7424,S0002181,30/06/1999,29/03/2005,1.0,1.00,percent,Nickel,10.0,kilotonne,NaN,NICKEL,10000.0,10000.0
7425,S0002181,30/06/1999,29/03/2005,1.0,0.07,percent,Cobalt,700.0,tonnes,NaN,NICKEL,700.0,700.0
7428,S0002182,30/06/1999,29/03/2005,0.4,0.14,percent,Cobalt,560.0,tonnes,NaN,NICKEL,560.0,1400.0


In [52]:
test_data=filtered_data(new_df)

In [53]:
test_data.head()

,SiteCode,Cobalt,Nickel
0,S0002179,900.0,10000.0
1,S0002181,700.0,10000.0
2,S0002182,1400.0,11000.0
3,S0002327,400.0,7500.0
4,S0002811,700.0,12000.0


In [54]:
test_data=test_data.loc[:, ['SiteCode','Nickel','Cobalt']] 

In [55]:
Sites_data = pd.read_csv('Sites.csv')

In [56]:
Sites_data=Sites_data.loc[:,['Latitude','Longitude','SiteCode']]

In [57]:
# Add Latitude, Longitude to dataframe
test_data=pd.merge(test_data,Sites_data,on='SiteCode',how='left')

In [58]:
test_data

,SiteCode,Nickel,Cobalt,Latitude,Longitude
0,S0002179,10000.0,900.0,-30.655910,121.826202
1,S0002181,10000.0,700.0,-30.736222,121.802770
2,S0002182,11000.0,1400.0,-30.772570,121.793602
3,S0002327,7500.0,400.0,-30.534769,122.084999
4,S0002327,7500.0,400.0,-30.534769,122.084999
5,S0002811,12000.0,700.0,-28.947189,121.786499
6,S0005015,9000.0,1300.0,-30.369413,121.146744
7,S0005039,11000.0,570.0,-28.797979,121.910699
8,S0005040,11100.0,560.0,-28.747169,121.895500
9,S0005041,10700.0,900.0,-28.821739,121.848602


In [59]:
original_test_data=test_data
data_pred_Co=test_data.loc[:, ['Nickel','Latitude','Longitude']] 

In [60]:
#Seperate dataset (exclude long and lat)
data_pred_Co=data_pred_Co.iloc[:,:-2]

In [61]:
data_pred_Co=data_pred_Co.replace(0,1)

In [62]:
data_pred_Co = data_pred_Co.apply(np.log, axis=1)

In [63]:
data_pred_Co['Latitude'] = original_test_data['Latitude']
data_pred_Co['Longitude'] = original_test_data['Longitude']

In [64]:
test_preds = np.exp(rf.predict(data_pred_Co))

In [65]:
result=original_test_data.loc[:,['SiteCode','Cobalt']]

In [66]:
result['Pred_Cobalt'] = test_preds


In [69]:
result

,SiteCode,Cobalt,Pred_Cobalt
0,S0002179,900.0,143.276409
1,S0002181,700.0,66.313210
2,S0002182,1400.0,344.066897
3,S0002327,400.0,179.775792
4,S0002327,400.0,179.775792
5,S0002811,700.0,535.672977
6,S0005015,1300.0,217.182031
7,S0005039,570.0,603.926617
8,S0005040,560.0,573.322654
9,S0005041,900.0,650.494628


### MSE from the mode based just on Nickel, longitude and latitude

In [70]:
print(f"Test MSE: {mean_squared_error(test_preds, result['Cobalt'])}")

Test MSE: 197869.37833977077


In [ ]:
# Model with trained by Nickel for predicting Cobalt has better MSE than the generalised model